In [2]:
# -*- coding: utf-8 -*-
"""MongoDB_1.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1Wi7YpIeFoaoUkhZWvS0NO83fyOjaREg0
"""

# Esta celda es solo para quienes utilizan Google colab
# -----------------------------------------------------

# Instalación de MongoDB
!apt update
!apt install wget curl gnupg2 software-properties-common apt-transport-https ca-certificates lsb-release
!curl -fsSL https://www.mongodb.org/static/pgp/server-6.0.asc|sudo gpg --dearmor -o /etc/apt/trusted.gpg.d/mongodb-6.gpg
!echo "deb [ arch=amd64,arm64 ] https://repo.mongodb.org/apt/ubuntu $(lsb_release -cs)/mongodb-org/6.0 multiverse" | tee /etc/apt/sources.list.d/mongodb-org-6.0.list
!apt update
!apt install mongodb-org
!mkdir /data
!mkdir /data/db
!mongod --fork --logpath /var/log/mongodb/mongod.log

!pip install pymongo

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,776 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,986 kB]
Get:13 http://archive.ubuntu.com/ubuntu ja

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import requests
from bs4 import BeautifulSoup
import json
import re

In [3]:
import pymongo
from pymongo import MongoClient

## Check server stats
!mongod --version

print ("MongoDB funcionando perfectamente. Versión:", pymongo.version)

# Creamos una conexión con el servidor usando la url de conexión por defecto.
url_server = 'localhost:27017'
client = MongoClient(url_server)


db version v6.0.24
Build Info: {
    "version": "6.0.24",
    "gitVersion": "1b052b94a23863fd12be97aaa4e4b1d96456e5cc",
    "openSSLVersion": "OpenSSL 3.0.2 15 Mar 2022",
    "modules": [],
    "allocator": "tcmalloc",
    "environment": {
        "distmod": "ubuntu2204",
        "distarch": "x86_64",
        "target_arch": "x86_64"
    }
}
MongoDB funcionando perfectamente. Versión: 4.13.1


In [13]:
def get_sp500_tickers():
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    soup = BeautifulSoup(requests.get(url).content, "html.parser")
    table = soup.find("table", {"id": "constituents"})
    tickers = []
    sectors = []
    for row in table.find_all("tr")[1:]:
        cols = row.find_all("td")
        tickers.append(cols[0].text.strip().replace(".", "-"))
        sectors.append(cols[3].text.strip())
    return pd.DataFrame({'Ticker': tickers, 'Sector': sectors})

def graham_number(eps, bvps):
    try:
        return np.sqrt(22.5 * eps * bvps) if eps > 0 and bvps > 0 else None
    except:
        return None

def get_stock_data(ticker):
    try:
        stock = yf.Ticker(ticker)
        info = stock.info

        pe_ratio = info.get("trailingPE", None)
        pb_ratio = info.get("priceToBook", None)
        eps = info.get("trailingEps", None)
        current_ratio = info.get("currentRatio", None)
        total_debt = info.get("totalDebt", None)
        total_equity = info.get("totalStockholderEquity", None)
        market_cap = info.get("marketCap", None)
        dividend_yield = info.get("dividendYield", None)
        book_value = info.get("bookValue", None)
        bvps = book_value
        graham_num = graham_number(eps, bvps)

        return {
            "Ticker": ticker,
            "P/E": pe_ratio,
            "P/B": pb_ratio,
            "EPS": eps,
            "BVPS": bvps,
            "Graham Number": graham_num,
            "Current Ratio": current_ratio,
            "Debt/Equity": total_debt / total_equity if total_debt and total_equity else None,
            "Market Cap": market_cap,
            "Dividend Yield": dividend_yield,
        }

    except:
        return None

# === MAIN SCRIPT ===

# Paso 1: obtener tickers y sectores
sp500_df = get_sp500_tickers()
sample_df = sp500_df.sample(50, random_state=42)  # usar 50 para no saturar Yahoo

# Paso 2: descargar datos financieros
datos = []
for _, row in sample_df.iterrows():
    info = get_stock_data(row["Ticker"])
    if info:
        info["Sector"] = row["Sector"]
        datos.append(info)

# Paso 3: guardar como JSON
with open("empresas_sp500_sample.json", "w") as f:
    json.dump(datos, f, indent=2)

# Descargar dividendos históricos reales
dividend_data = []

for ticker in sample_df['Ticker']:
    stock = yf.Ticker(ticker)
    dividends = stock.dividends
    if not dividends.empty:
        for date, amount in dividends.items():
            dividend_data.append({
                "Ticker": ticker,
                "Fecha": str(date.date()),
                "Monto": float(round(amount, 4))
            })

# Guardar como archivo JSON
with open("dividendos_sp500_sample.json", "w") as f:
    json.dump(dividend_data, f, indent=2)


In [14]:
# Paso 4: importar a MongoDB
# client = MongoClient("mongodb://localhost:27017/")
db = client["graham"]
empresas = db["empresas"]
empresas.drop()  # limpiar antes de insertar
empresas.insert_many(datos)

# Importar dividendos
dividendos = db["dividendos"]

with open("dividendos_sp500_sample.json") as f:
    datos = json.load(f)
    dividendos.insert_many(datos)

# Paso 5: realizar una consulta
# pipeline = [
#     {"$match": {"P/E": {"$lt": 15}, "Dividend Yield": {"$gt": 0.02}}},
#     {"$group": {"_id": "$Sector", "prom_eps": {"$avg": "$EPS"}}}
# ]

pipeline = [
    {"$match": {"P/E": {"$lt": 15}}},
    {"$lookup": {
        "from": "dividendos",
        "localField": "Ticker",
        "foreignField": "Ticker",
        "as": "HistorialDividendos"
    }},
    {"$unwind": "$HistorialDividendos"},
    {"$group": {
        "_id": "$Ticker",
        "avg_dividendo": {"$avg": "$HistorialDividendos.Monto"},
        "eps": {"$first": "$EPS"},
        "sector": {"$first": "$Sector"}
    }},
    {"$sort": {"avg_dividendo": -1}}
]


resultados = list(empresas.aggregate(pipeline))
for r in resultados:
    print(r)

{'_id': 'CB', 'avg_dividendo': 0.39783023255813954, 'eps': 20.77, 'sector': 'Property & Casualty Insurance'}
{'_id': 'VICI', 'avg_dividendo': 0.3464137931034483, 'eps': 2.5, 'sector': 'Hotel & Resort REITs'}
{'_id': 'BG', 'avg_dividendo': 0.33695744680851064, 'eps': 7.79, 'sector': 'Agricultural Products & Services'}
{'_id': 'HIG', 'avg_dividendo': 0.26864406779661015, 'eps': 10.03, 'sector': 'Property & Casualty Insurance'}
{'_id': 'MRK', 'avg_dividendo': 0.2265952, 'eps': 6.88, 'sector': 'Pharmaceuticals'}
{'_id': 'NEM', 'avg_dividendo': 0.150443949044586, 'eps': 4.39, 'sector': 'Gold'}
{'_id': 'UHS', 'avg_dividendo': 0.09895348837209302, 'eps': 17.8, 'sector': 'Health Care Facilities'}
